In [2]:
%cd code
import os, sys
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from torchvision import transforms
from my_dataset import TestDataset
from model import AttU_Net
from model import init_weights
from mask_to_submission import masks_to_submission
import train_helpers as th


%run -i my_dataset.py
%run -i model.py
%run -i train_helpers.py
%run -i mask_to_submission.py




/Users/mverest/Desktop/clean_ML_repo/code


In [5]:
TEST_IMG_DIR = "Data/test_set_images"
MODEL_PATH = "saved_models\VBest_best.pth"
THRESHOLD = 0.5
TEST_IMG_DIR = "predictions"
SUBMISSION_FILE_NAME = 'Best_submission.csv'

In [ ]:


test_transform = A.Compose([
    A.Normalize(mean=[0.0, 0.0, 0.0], 
                std=[1.0, 1.0, 1.0], 
                max_pixel_value=255.0),
    ToTensorV2(),
                ])

test_dataset = TestDataset(root_dir=TEST_IMG_DIR, 
                           transform=test_transform, 
                           patch_size= None, 
                           stride= None, 
                           CLAHE= True)

test_loader = DataLoader(test_dataset, 
                         shuffle=False,
                         batch_size=1)


    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AttU_Net().to(device)
init_weights(model, init_type='kaiming', gain=0.02)

model.load_state_dict(torch.load(MODEL_PATH, map_location=device, weights_only=True))

predicted_mask = th.prediction_on_test_set(model=model, test_loader=test_loader, device=device, threshold=THRESHOLD, )


image_filenames = []
for i in range(1, 51):
    image_filename = os.path.join(TEST_IMG_DIR,f"test_{i}_mask" + '.png')
    print(image_filename)
    image_filenames.append(image_filename)
masks_to_submission(SUBMISSION_FILE_NAME, *image_filenames)










